In [1]:
! pip install langchain langchain_google_genai
! pip install pinecone
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 558.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.6 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [2]:
! pip install langchain_pinecone
! pip install langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 716.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 982.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.1/221.1 kB 2.1 MB/s eta 0:00:00


In [25]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore # Import PineconeVectorStore
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory
import pinecone
import getpass
# Import the Pinecone library
from pinecone import Pinecone
from langchain_huggingface import HuggingFaceEmbeddings # new update in langchain
from pinecone import Pinecone, ServerlessSpec
from time import time

In [2]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [4]:
if "PINECONE_API_KEY" not in os.environ:
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

In [21]:
PINECONE_API_KEY="Write/Paste your key here"

In [7]:
#make a dir pdf of papers

if not os.path.exists("papers"):
    os.makedirs("papers")

In [8]:
directory ="/content/papers"

In [9]:
papers=os.listdir(directory)
papers

['Weakly_Supervised_Deep_Learning_for_the_Detection_of_Domain_Generation_Algorithms.pdf',
 'deep fake weak supervise learning - Consensus.pdf',
 'journal.pone.0326565.pdf']

In [10]:
print("total number of PDFs:", len(papers))

total number of PDFs: 3


In [11]:
INDEX_NAME = "rag-pdf-index"  # Pinecone index name

In [12]:
EMBEDDING_MODEL = "sentence-transformers/all-mpnet-base-v2"  # Better embedding model

In [16]:
def initialize_pinecone():
    """Initialize Pinecone and create index if needed"""
    try:
        # Initialize Pinecone client with explicit API key
        pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

        # List existing indexes
        existing_indexes = pc.list_indexes().names()
        print(f"Existing indexes: {existing_indexes}")

        # Create index if it doesn't exist
        if INDEX_NAME not in existing_indexes:
            print(f"Creating new index: {INDEX_NAME}")
            pc.create_index(
                name=INDEX_NAME,
                dimension=768,  # Match embedding dimension for all-mpnet-base-v2
                metric="cosine",
                spec=ServerlessSpec(
                    cloud="aws",
                    region="us-east-1"
                )
            )
            print(f"Index {INDEX_NAME} created successfully")
            # Wait for index to be ready
            time.sleep(10)
        else:
            print(f"Using existing index: {INDEX_NAME}")

        return pc.Index(INDEX_NAME)

    except Exception as e:
        print(f"Error initializing Pinecone: {e}")
        print("Please check your Pinecone API key and try again.")
        return None

In [16]:
! pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 1.0 MB/s eta 0:00:00


In [14]:
import re
def clean_filename(path):
    """Extract clean filename from path"""
    filename = os.path.basename(path)
    return re.sub(r'[^a-zA-Z0-9_.-]', ' ', filename)

In [17]:
def load_and_process_documents():
    """Load and process PDF documents"""
    # Check if papers directory has files
    papers = os.listdir(directory)
    pdf_files = [f for f in papers if f.lower().endswith('.pdf')]

    if not pdf_files:
        print(f"No PDF files found in {directory}")
        print("Please upload PDF files to the papers directory.")
        return None, None

    print(f"Found {len(pdf_files)} PDF files: {pdf_files}")

    # Load documents
    loader = PyPDFDirectoryLoader(directory, recursive=True)
    raw_documents = loader.load()
    print(f"Loaded {len(raw_documents)} pages from PDFs")

    # Clean metadata
    documents = []
    for doc in raw_documents:
        clean_source = clean_filename(doc.metadata['source'])
        doc.metadata.update({
            'source': clean_source,
            'filename': clean_source
        })
        documents.append(doc)

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200,
        chunk_overlap=150,
        length_function=len
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split into {len(chunks)} text chunks")

    return documents, chunks

In [18]:
def create_vector_store(chunks):
    """Create vector store with embeddings"""
    # Initialize embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL,
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )
    print("Embeddings model loaded")

    # Initialize Pinecone
    pinecone_index = initialize_pinecone()
    if pinecone_index is None:
        return None

    try:
        # Create vector store
        vector_store = PineconeVectorStore.from_documents(
            chunks,
            embeddings,
            index_name=INDEX_NAME,
            pinecone_api_key=os.environ["PINECONE_API_KEY"]
        )
        print("Documents embedded and stored in Pinecone successfully")
        return vector_store

    except Exception as e:
        print(f"Error creating vector store: {e}")
        return None

In [19]:
def setup_qa_chain(vector_store):
    """Setup the conversational QA chain"""
    # Create retriever
    retriever = vector_store.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": 5,
            "score_threshold": 0.7,
            "filter": {"source": {"$exists": True}}
        }
    )

    # Initialize Gemini LLM
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0.6,
        max_output_tokens=1500,
        top_k=40,
        top_p=0.95
    )

    # Create conversation memory
    memory = ConversationBufferWindowMemory(
        memory_key="chat_history",
        k=3,
        return_messages=True,
        output_key="answer"
    )

    # Create conversational RAG chain
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        memory=memory,
        return_source_documents=True,
        get_chat_history=lambda h: h
    )

    return qa_chain

In [23]:
def main():
    """Main function to set up the RAG system"""
    print("Setting up RAG system...")

    # Step 1: Load and process documents
    documents, chunks = load_and_process_documents()
    if chunks is None:
        return

    # Step 2: Create vector store
    vector_store = create_vector_store(chunks)
    if vector_store is None:
        return

    # Step 3: Setup QA chain
    qa_chain = setup_qa_chain(vector_store)

    print("\n" + "="*50)
    print("RAG system ready! You can now ask questions about your documents.")
    print("Type 'bye', 'exit', or 'quit' to end the conversation.")
    print("="*50)

    # Interactive chat loop
    while True:
        try:
            query = input("\nYou: ")

            if query.lower().strip() in ["bye", "exit", "quit", ""]:
                print("Assistant: Goodbye! Have a great day!")
                break

            # Process the query
            print("Assistant: Thinking...")
            result = qa_chain({"question": query})

            # Display response
            print(f"\nAssistant: {result['answer']}")

            # Show sources if requested
            if any(word in query.lower() for word in ["source", "reference", "cite", "paper"]):
                print("\nSources:")
                for i, doc in enumerate(result["source_documents"][:3]):
                    source = doc.metadata.get('source', 'Unknown')
                    page = doc.metadata.get('page', 'N/A')
                    print(f"{i+1}. {os.path.basename(source)} (Page {page})")
                    print(f"   Excerpt: {doc.page_content[:150]}...\n")

        except KeyboardInterrupt:
            print("\nAssistant: Goodbye! Have a great day!")
            break
        except Exception as e:
            print(f"\nError: {e}")
            print("Please try again with a different question.")

# Run the main function
if __name__ == "__main__":
    main()

Setting up RAG system...
Found 3 PDF files: ['Weakly_Supervised_Deep_Learning_for_the_Detection_of_Domain_Generation_Algorithms.pdf', 'deep fake weak supervise learning - Consensus.pdf', 'journal.pone.0326565.pdf']
Loaded 58 pages from PDFs
Split into 195 text chunks
Embeddings model loaded
Existing indexes: ['rag-pdf-index']
Using existing index: rag-pdf-index


/tmp/ipython-input-19-2749410998.py:23: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


Documents embedded and stored in Pinecone successfully

RAG system ready! You can now ask questions about your documents.
Type 'bye', 'exit', or 'quit' to end the conversation.

You: """     Based on all the research papers you have access to, write a comprehensive literature review paragraph      in IEEE citation style. The paragraph should:          1. Summarize the main contributions of each paper     2. Highlight the pros and cons of different approaches     3. Identify research gaps and limitations     4. Show the progression of ideas in the field     5. Be written in academic style suitable for a research paper          Please provide a well-structured literature review paragraph that synthesizes      the findings from all available papers.     """
Assistant: Thinking...


/tmp/ipython-input-23-4217766465.py:34: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"question": query})



Assistant: Recent research has explored weakly supervised deep learning for various tasks.  Varma et al. [1]  pioneered the use of deep reinforcement learning for anomaly detection in business processes, effectively leveraging limited labeled data, but the specifics of its generalizability remain unclear.  Elaziz et al. [2] further advanced this approach, demonstrating improved performance compared to competing methods by exploring unlabeled data.  Karamanolakis et al. [3] introduced ASTRA, a self-training framework that enhanced text classification by effectively utilizing all available data, showcasing the potential of weak supervision for improving deep neural network performance.  Zhao et al. [4] applied weakly supervised deep embedding to product review sentiment analysis, effectively classifying review sentences using ratings as weak supervision signals.  While these studies demonstrate the effectiveness of weakly supervised deep learning across diverse applications, limitations

Assistant: Recent research has explored weakly supervised deep learning for various tasks.  Varma et al. [1]  pioneered the use of deep reinforcement learning for anomaly detection in business processes, effectively leveraging limited labeled data, but the specifics of its generalizability remain unclear.  Elaziz et al. [2] further advanced this approach, demonstrating improved performance compared to competing methods by exploring unlabeled data.  Karamanolakis et al. [3] introduced ASTRA, a self-training framework that enhanced text classification by effectively utilizing all available data, showcasing the potential of weak supervision for improving deep neural network performance.  Zhao et al. [4] applied weakly supervised deep embedding to product review sentiment analysis, effectively classifying review sentences using ratings as weak supervision signals.  While these studies demonstrate the effectiveness of weakly supervised deep learning across diverse applications, limitations exist.  The reliance on weak supervision signals can introduce noise and bias [2, 4], and the generalizability of these models across different datasets and tasks requires further investigation [1, 3].  Future research should focus on developing more robust and generalizable weakly supervised learning methods that mitigate the effects of noisy labels and improve the efficiency of data utilization.  Furthermore, exploring the integration of different weak supervision strategies and developing techniques for evaluating the quality of weak supervision signals are crucial for advancing the field.

[1] P. Varma et al., “Deep reinforcement learning for data-efficient weakly supervised business process anomaly detection,” *Proceedings of the VLDB Endowment*, vol. 7, 2019.
[2] E. A. Elaziz et al., “Journal of Big Data,” 2023.
[3] G. Karamanolakis et al., “Self-Training with Weak Supervision,” *ArXiv*, 2021.
[4] W. Zhao et al., “Weakly-Supervised Deep Embedding for Product Review Sentiment Analysis,” *IEEE Transactions on Knowledge and Data Engineering*, 2018.

Assistant: The provided text mentions AUC (Area Under the Curve) scores as an evaluation metric in one study,  reporting scores ranging from 44.45% to 72.69% on prospective data.  However, it does not offer benchmark accuracies for comparison across all cited studies.  Several papers mention achieving "convincing performance" or outperforming competing approaches, but lack specific numerical benchmark accuracies.


Assistant: The research paper "Single-layer KAN for deepfake classification: Balancing efficiency and performance in resource-constrained environments" introduces a single-layer Kolmogorov-Arnold Network (KAN) for deepfake detection.  The model uses a SiLU activation function and a spline-based transformation, aiming for efficiency while maintaining performance.  The study compares the KAN model to benchmark Convolutional Neural Networks (CNNs), highlighting its advantages in terms of memory usage and computational efficiency, particularly relevant for resource-constrained environments.  The KAN model was trained on the Celeb-DF and FaceForensics++ datasets, and its performance is evaluated and compared to other methods, including DefakeHop, which achieved 90.56% accuracy with 42.8K parameters.  The paper also discusses the use of the Augur tool for assessing the computational requirements of CNNs on edge devices.
